In [ ]:
import pandas as pd
from redblackgraph.reference.avos import avos_sum, avos_product
labels = [-1,0,1,2,3,4,5,6,7,8,9]

# Avos Algebraic Structure

This notebook explores a novel algebraic structure that emerges from modeling genealogical relationships. While initially conceived as an extension of linear algebra, **this structure does NOT form a classical vector space**. Instead, it is a **domain-specific algebraic structure** that is well-suited for representing ancestry and descendancy relationships.

## Why Not a Vector Space?

A vector space over a field F must satisfy 8 axioms. The Avos structure **fails several of these**:

### Axiom Failures

| Axiom | Status | Reason |
|-------|--------|--------|
| 1. Associativity of Addition | ✓ | `avos_sum` is associative |
| 2. Commutativity of Addition | ✓ | `avos_sum` is commutative |
| 3. Additive Identity | ✓ | 0 acts as identity (treated as ∞ in min operation) |
| 4. Additive Inverse | ✗ | No element `-a` exists such that `a ⊕ (-a) = 0` |
| 5. Compatibility of Scalar Mult | ✗ | `avos_product` is NOT associative with identity values |
| 6. Multiplicative Identity | ✗ | No single identity; RED_ONE and BLACK_ONE are parity-dependent |
| 7. Distributivity (scalar over vector) | ✓ | Holds for non-identity values |
| 8. Distributivity (field over scalar) | ✗ | Fails due to parity constraints |

### The Underlying "Field" is Not a Field

The scalars (integers with avos operations) don't form a field:
- No multiplicative inverses (can't divide)
- No additive inverses (avos_sum has no negation)
- Multiplication is not associative when identities involved

## What Structure IS This?

The Avos structure is best characterized as a **module over a near-semiring** or simply a **domain-specific algebraic structure** for genealogical modeling.

It shares properties with:
- **Tropical (min-plus) algebra**: Uses min for addition
- **Two-sorted algebras**: Separate behavior for even/odd (male/female)
- **Category theory**: Objects are parity classes, morphisms are relationships

## Scalar Addition (Avos Sum)

Scalar addition is the *non-zero minimum* operator:

$$a \oplus b = \begin{cases}
        b, & \text{if }a=0,\\
        a, & \text{if }b=0,\\
        \min(a, b), & \text{otherwise}
\end{cases}$$

The element 0 acts as the **additive identity** (conceptually representing "no relationship" or ∞ distance).

In [ ]:
pd.DataFrame(data=( [ [avos_sum(i, j) for j in range(-1, 10)] for i in range(-1, 10)] ), index=labels, columns=labels)

## Scalar Multiplication (Avos Product)

For positive integers > 1, the avos product operates on the binary representation: it replaces the leftmost significant bit of the RHS operand with the value of the LHS operand.

This is symbolically represented by $\otimes$, e.g., $5 \otimes 2 = 10$.

**Note:** The avos product is **NOT commutative** and **NOT fully associative** (see below).

### Binary Representation Examples

The bit-shift operation is easier to understand with binary representations:

| $a$ | $b$ | $a_{2}$ | $b_{2}$ | $(a \otimes b)_{2}$ | $a \otimes b$ |
|-----|-----|---------|---------|---------------------|---------------|
| 1   | 2   | 1       | 10      | 10                  | 2             |
| 2   | 1   | 10      | 1       | 10                  | 2             |
| 2   | 2   | 10      | 10      | 100                 | 4             |
| 2   | 3   | 10      | 11      | 101                 | 5             |
| 3   | 2   | 11      | 10      | 110                 | 6             |
| 3   | 3   | 11      | 11      | 111                 | 7             |
| 2   | 4   | 10      | 100     | 1000                | 8             |
| 7   | 4   | 111     | 100     | 11100               | 28            |

The pattern: replace the leading 1 in $b$ with all the bits of $a$.

### Formal Definition (for values > 1)

$$a \otimes b = (b \land (2^{MSB(b)} - 1)) \lor (a \ll MSB(b))$$

where MSB(x) is the position of the most significant bit of x.

### Zero Property
$$0 \otimes a = a \otimes 0 = 0$$

### Parity Identity Constraints

The special values RED_ONE (-1) and BLACK_ONE (1) have **asymmetric, parity-dependent behavior**:

**Left operand (starting point):**
- `RED_ONE ⊗ value = value` (male starting point, no filtering)
- `BLACK_ONE ⊗ value = value` (female starting point, no filtering)

**Right operand (gender filter):**
- `value ⊗ RED_ONE = value` if value is even, else 0 (filter for males)
- `value ⊗ BLACK_ONE = value` if value is odd, else 0 (filter for females)

**Identity × Identity:**
- `RED_ONE ⊗ RED_ONE = RED_ONE` (male self-loop)
- `BLACK_ONE ⊗ BLACK_ONE = BLACK_ONE` (female self-loop)
- `RED_ONE ⊗ BLACK_ONE = 0` (cross-gender undefined)
- `BLACK_ONE ⊗ RED_ONE = 0` (cross-gender undefined)

### Associativity Failure

Due to parity constraints, associativity **fails** when identities appear in middle positions:

```
(2 ⊗ BLACK_ONE) ⊗ 3 = 0 ⊗ 3 = 0
2 ⊗ (BLACK_ONE ⊗ 3) = 2 ⊗ 3 = 5
```

This is intentional for genealogical semantics but violates classical algebraic axioms.

In [ ]:
pd.DataFrame(data=( [ [avos_product(i, j) for j in range(-1, 10)] for i in range(-1, 10)] ), index=labels, columns=labels)

## Vector Operations for Genealogical Modeling

Each vector represents a person and their ancestry/descendancy. Vectors are either:
- **Red** (has one element with value -1): represents a male
- **Black** (has one element with value 1): represents a female

The "self element" $v_s$ denotes the diagonal element indicating the vertex's own identity.

### Constraint: Temporal/Biological Validity

Operations that would create impossible relationships (cycles, same-gender parents) result in $\vec{0}$.

### Offspring Composition

When two individuals produce offspring, we compose their ancestry vectors. This is **NOT a cross product** in the linear algebra sense (which produces an orthogonal vector). Instead, it's a **genealogical composition** that combines ancestries.

**Constraints:** 
- $\alpha_{s_\beta} = 0$ (α is not an ancestor of β)
- $\beta_{s_\alpha} = 0$ (β is not an ancestor of α)  
- $\alpha_s \neq \beta_s$ (different genders)

**Formula:**
$$\vec{\text{offspring}} = 2\vec{\alpha} \oplus 3\vec{\beta}$$

where the offspring's self-element is 1 (female) or -1 (male), and $\oplus$ denotes element-wise avos_sum.

The factors 2 and 3 encode "father" and "mother" relationships respectively in the pedigree numbering scheme.

### Adding a Relationship (Edge Composition)

When adding a new ancestral relationship to an existing vector:

**Constraints:**
- $\alpha_{s_\beta} = 0$ (no existing relationship)
- $\beta_{s_\alpha} = 0$ (no reverse relationship that would create cycle)

**Formula:**
$$\vec{\alpha'} = \vec{\alpha} \oplus r \cdot \vec{\beta}$$

where $r$ is the relationship value:
- $r = 2$ if $\vec{\beta}$ is red (adding a male ancestor)
- $r = 3$ if $\vec{\beta}$ is black (adding a female ancestor)

The scalar multiplication $r \cdot \vec{\beta}$ shifts β's ancestry to the correct position in α's pedigree, and the avos_sum combines them (taking shortest paths).

### Relational Composition in Matrix Form

These vector operations correspond to the **vertex relational composition** and **edge relational composition** operations defined in the main Red Black Graph notebook. They allow efficient incremental updates to the transitive closure matrix $R^+$.

## Structural Constraints

The Avos algebraic structure has inherent constraints that ensure well-formed genealogical representations.

### Parity Classes

Values are partitioned into two parity classes based on their genealogical meaning:

| Class | Values | Identity | Represents |
|-------|--------|----------|------------|
| **Even (Male)** | RED_ONE (-1), 2, 4, 6, 8, ... | RED_ONE | Paternal lineage |
| **Odd (Female)** | BLACK_ONE (1), 3, 5, 7, 9, ... | BLACK_ONE | Maternal lineage |
| **Zero** | 0 | — | No relationship (absorbing element) |

The parity encodes which gender the ancestor is:
- Father = 2 (even), Mother = 3 (odd)
- Paternal grandfather = 4 (even), Paternal grandmother = 5 (odd)
- Maternal grandfather = 6 (even), Maternal grandmother = 7 (odd)

### Vector Element Constraints

For any vector $\vec{v}$ representing a person's ancestry:

1. **Uniqueness**: Each non-zero value appears **at most once** across all elements
   - Rationale: Each ancestor occupies exactly one position in a pedigree chart

2. **Self-element**: Exactly one element contains the identity (RED_ONE or BLACK_ONE)
   - This is the diagonal element $v_s$ representing the person's own gender

3. **Parity consistency**: The parity of each non-zero element must match the parity of the ancestor at that position
   - A person's father is always at an even-valued position
   - A person's mother is always at an odd-valued position

### Dimensional Constraints (Matrix Form)

When vectors are arranged as rows in a matrix $R$:

1. **Column parity**: Each column $j$ represents a specific person with fixed gender
   - Column values must be consistent with that person's parity

2. **Diagonal**: $R_{ii}$ = RED_ONE (-1) if person $i$ is male, BLACK_ONE (1) if female

3. **Closure**: The transitive closure $R^+$ maintains all constraints
   - No new constraint violations can be introduced by path composition

# Comparison with Classical Linear Algebra

## What Carries Over

Despite not being a vector space, several linear algebra concepts apply:

| Concept | Classical | Avos Structure |
|---------|-----------|----------------|
| Matrix-vector product | Sum of products | Avos sum of avos products |
| Matrix multiplication | Row × column dot products | Row × column avos products |
| Transitive closure | $A^* = I + A + A^2 + ...$ | $R^+ = R \otimes R \otimes ... $ (converges in ≤n steps) |
| Shortest paths | Dijkstra, Floyd-Warshall | Modified Floyd-Warshall with avos ops |
| Linear independence | Zero combination test | Modified (see below) |
| Rank | Dimension of column space | Number of independent ancestry lines |

## Linear Independence in Avos Structure

Linear independence **does apply**, but with interesting properties due to the nature of avos_sum.

### Classical Definition (Adapted)

Vectors $\vec{v_1}, \vec{v_2}, ..., \vec{v_n}$ are **Avos-independent** if:

$$c_1 \otimes \vec{v_1} \oplus c_2 \otimes \vec{v_2} \oplus ... \oplus c_n \otimes \vec{v_n} = \vec{0}$$

implies all $c_i = 0$.

### Key Insight: No Cancellation

Since avos_sum is the non-zero minimum:
- `a ⊕ b = 0` **only if** both `a = 0` AND `b = 0`
- You cannot "cancel" non-zero values (there's no subtraction)

This means **any set of non-zero vectors satisfies the zero-combination test**—you cannot produce $\vec{0}$ from non-zero vectors with non-zero scalars.

### More Useful: Ancestry Independence

In the genealogical context, a richer notion of independence emerges:

**Ancestry-dependent**: Vector $\vec{u}$ depends on $\vec{v}$ if $v$'s ancestry is "contained within" $u$'s ancestry (i.e., $v$ is an ancestor of $u$).

**Ancestry-independent**: A set of vectors where no vector's subject is an ancestor of another's.

**Siblings are dependent**: Two siblings share identical ancestry (same parents), so their row vectors in $R^+$ are identical except for the self-element. They are ancestry-dependent because they share common ancestors.

**Unrelated individuals are independent**: If two people share no common ancestors, their row vectors have no overlapping non-zero elements (except possibly 0s).

### Nullspace Interpretation

In classical LA, vectors are independent if the nullspace of matrix $A$ (with vectors as columns) is only $\{\vec{0}\}$.

In Avos structure:
- The "nullspace" concept applies but is trivial for non-zero vectors
- More meaningful: the **reachability structure** determines dependencies
- If $R^+_{ij} \neq 0$, row $i$ "depends on" column $j$ (there's a path)

## What Doesn't Apply Directly

- **Eigenvalues/Eigenvectors**: Require field properties (division, additive inverse)
- **Determinants**: Require subtraction
- **Orthogonality**: No inner product structure with the required properties

## Connection to Tropical Geometry

The Avos structure is related to **tropical (min-plus) algebra**:
- Both use min for "addition"
- Both use a different operation for "multiplication"
- Both form semirings (not rings or fields)
- Both have modified notions of linear algebra (tropical linear algebra is an active research area)

Tropical geometry has applications in optimization, phylogenetics, and algebraic geometry. The Avos extension adds parity/gender tracking for genealogical applications.

## References

- Main notebook: "Red Black Graph - A DAG of Multiple, Interleaved Binary Trees"
- Mathematical analysis: `MATHEMATICAL_ANALYSIS.md` in repository root
- Related: Tropical geometry, min-plus algebra, semiring theory, tropical linear algebra